In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
cust_data = pd.read_csv('data_transform.csv')
cust_data.shape

(100233, 126)

In [3]:
cust_data.head()

,ACTL_FMLY_NUM,AGE,ARPU,AUTR_FAIL_MCNT,AVG_CALL_FREQ,AVG_CALL_TIME,AVG_STLN_RATE,CB_GUIF_AMT,CB_GUIF_CNT,CNTT_LAMT_CNT,...,MATE_OCCP_NAME_C_07,MATE_OCCP_NAME_C_08,MATE_OCCP_NAME_C_09,MATE_OCCP_NAME_C_10,MATE_OCCP_NAME_C_11,MATE_OCCP_NAME_C_12,MATE_OCCP_NAME_C_13,MATE_OCCP_NAME_C_14,MATE_OCCP_NAME_C_15,MATE_OCCP_NAME_C_16
0,4,50,30000,10,493,450,0,420001,3,0,...,0,0,0,0,0,0,0,0,1,0
1,4,50,30000,0,22,81,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,4,60,30000,0,17,139,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,35,30000,0,0,1118,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,45,50000,0,354,396,95,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
def model_performance(y_test, y_pred):    
    print('confusion matrix')
    print(metrics.confusion_matrix(y_test, y_pred))
    print('accuracy : {}'.format(metrics.accuracy_score(y_test, y_pred).round(3)))
    print('precision : {}'.format(metrics.precision_score(y_test, y_pred, pos_label=1).round(3)))
    print('recall : {}'.format(metrics.recall_score(y_test, y_pred, pos_label=1).round(3)))
    print('F1 : {}'.format(metrics.f1_score(y_test, y_pred, pos_label=1).round(3)))

## 연체자 data duplicate

In [5]:
cust = cust_data[cust_data['TARGET'] == 0]
cust_overdue = cust_data[cust_data['TARGET'] == 1]     # 연체자

print(cust.shape[0])
print(cust_overdue.shape[0])

95946
4287


In [6]:
variables = cust_data.drop('TARGET', 1).columns
variables

Index(['ACTL_FMLY_NUM', 'AGE', 'ARPU', 'AUTR_FAIL_MCNT', 'AVG_CALL_FREQ',
       'AVG_CALL_TIME', 'AVG_STLN_RATE', 'CB_GUIF_AMT', 'CB_GUIF_CNT',
       'CNTT_LAMT_CNT',
       ...
       'MATE_OCCP_NAME_C_07', 'MATE_OCCP_NAME_C_08', 'MATE_OCCP_NAME_C_09',
       'MATE_OCCP_NAME_C_10', 'MATE_OCCP_NAME_C_11', 'MATE_OCCP_NAME_C_12',
       'MATE_OCCP_NAME_C_13', 'MATE_OCCP_NAME_C_14', 'MATE_OCCP_NAME_C_15',
       'MATE_OCCP_NAME_C_16'],
      dtype='object', length=125)

### ML modeling

In [10]:
# train / test set 분리
x = cust_data.drop('TARGET', axis=1)
y = cust_data['TARGET']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

print('연체자 수 train : {} / {}'.format(sum(y_train == 1), y_train.shape[0]))
print('연체자 수 test  : {} / {}'.format(sum(y_test  == 1), y_test.shape[0]))

연체자 수 train : 3439 / 80186
연체자 수 test  : 848 / 20047


In [11]:
# train set 중에서 연체자만 추출
x_overdue = x_train[y_train == 1]
y_overdue = y_train[y_train == 1]
print(x_overdue.shape)
print(y_overdue.shape)

(3439, 125)
(3439,)


In [12]:
max_epoch = 9

for i in range(max_epoch+1):
    x2_train, x2_test, y2_train, y2_test = train_test_split(x_train, y_train, test_size=0.2, random_state=0)
    
    print('========================================= ', i)
#     print('test {} : test overdue {}'.format(y2_test.shape[0], sum(y2_test == 1)))
#     print('')
    
    clf = DecisionTreeClassifier()
#     clf = GradientBoostingClassifier(n_estimators=500, random_state=0)
    
    clf.fit(x2_train, y2_train)
    
    y_pred = clf.predict(x2_test)
    model_performance(y2_test, y_pred)
    
    varDic = {'var':variables, 'importance':clf.feature_importances_}
    importance = pd.DataFrame(varDic)
    print(importance.sort_values(by='importance', ascending=False).head(10))

    x_train = pd.concat([x_train, x_overdue])
    y_train = pd.concat([y_train, y_overdue])

#     if i == max_epoch:
#     print('------------------------')
#     y_pred_0 = clf.predict(x_test)
#     model_performance(y_test, y_pred_0)
    print('------------------------')

=========================================  0
confusion matrix
[[14757   561]
 [  523   197]]
accuracy : 0.932
precision : 0.26
recall : 0.274
F1 : 0.267
    importance              var
78    0.062590      PAYM_METD_G
14    0.053831    CRDT_CARD_CNT
54    0.043811  SPTCT_OCCR_MDIF
17    0.031038   CRDT_OCCR_MDIF
58    0.026946     TEL_CNTT_QTR
49    0.024513        MOBL_PRIN
64    0.024307     TOT_LOAN_CNT
4     0.024043    AVG_CALL_FREQ
34    0.023324   HSHD_INFR_INCM
63    0.021932     TOT_LNIF_AMT
------------------------
=========================================  1
confusion matrix
[[14667   643]
 [  204  1211]]
accuracy : 0.949
precision : 0.653
recall : 0.856
F1 : 0.741
    importance              var
52    0.105835   OVDU_HIGH_RATE
14    0.071197    CRDT_CARD_CNT
54    0.042488  SPTCT_OCCR_MDIF
17    0.033798   CRDT_OCCR_MDIF
49    0.032815        MOBL_PRIN
64    0.030066     TOT_LOAN_CNT
34    0.028847   HSHD_INFR_INCM
63    0.023212     TOT_LNIF_AMT
1     0.023045              